In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import datasets
from tensorflow.keras.applications.resnet50 import ResNet50

#학습 데이터셋 가져오기
train_datasets = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/fish',
    image_size = (250,250),
    batch_size = 64,
    subset = 'training',
    validation_split=0.01,
    label_mode='categorical',
    seed = 1234
)

#테스트 데이터셋 가져오기
test_datasets = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/fish',
    image_size = (250,250),
    batch_size = 64,
    subset = 'validation',
    validation_split=0.01,
    label_mode='categorical',
    seed = 1234
)

# ResNet50 불러오기
base_model = ResNet50(include_top=False, pooling = 'avg' , input_shape = (250,250 ,3), weights = 'imagenet') #input shape 수정

#학습 못 하게 하기
for i in base_model.layers :
  i.trainable = False


#모델 layer 설계
reshape_layer = tf.keras.layers.Reshape((1, 1, 2048))(base_model.output)
x = tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal')(reshape_layer)
x = tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)(x)
x = tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(135, activation='softmax')(x)


model_res = tf.keras.Model(base_model.input, output)


# 모델 컴파일
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model_res.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy']) 
history = model_res.fit(train_datasets,validation_data=test_datasets, epochs=35) #학습하기



Found 18953 files belonging to 135 classes.
Using 18764 files for training.
Found 18953 files belonging to 135 classes.
Using 189 files for validation.
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 250, 250, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 256, 256, 3)  0           ['input_10[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 125, 125, 64  9472        ['conv1_pad[0][0]']              
                                )     

In [3]:
#테스트 데이터셋 테스트 코드

import random

class_name = ['가드네리 킬리피쉬', '고도비', '골든 구라미', '골든 바브', '골든 알지이터', '구피', '그린 네온 테트라', '글라스 블러드 핀 테트라', '글라스캣', '글로라이트 다니오', 
              '글로라이트 테트라', '나비 비파', '난주 금붕어', '남미 복어', '네온 드워프 레인보우', '네온 테트라', '뉴기니아 레인보우', '다람쥐 시클리드', '다이아몬드 테트라', 
              '드워프 구라미', '디스커스', '라스보라 갤럭시', '라스보라 에메랄드', '라스보라 쿠보타이', '라스보라 헤테로몰파', '라스보라 헹겔리', '라이어테일 몰리', '러미노즈 테트라', 
              '레드 레인보우', '레드 플래티', '레몬 테트라', '레오파드 다니오', '레인보우 샤크', '로즈라인 바브', '로지 바브', '리코리스 구라미', '마다가스카르 레인보우', '메티니스', 
              '메티니스 블랙바', '몽크호샤 코스테', '몽크호샤(레드아이 테트라)', '물티', '미키마우스 플래티', '바나나 시클리드', '백설 시클리드', '백운산', '밴디드 구라미', '밴디드 레인보우', 
              '밴디드 레포리너스', '버터플라이 레인보우', '벌룬 라미레지', '범블비 플래티', '베일테일 베타', '보세마니 레인보우', '볼리비안 라미레지', '브론즈 코리도라스', '브리샤르디', 
              '블랙 네온 테트라', '블랙 테트라(컬러 테트라)', '블루 레인보우', '블루 스팟 구라미', '블루 제브라 시클리드', '블루킹 테트라', '비너스투스 시클리드', '뽀본데타 레인보우', '샤페 테트라', 
              '선셋 구라미', '선셋 플래티', '세베럼', '셀핀 몰리', '소드테일', '수마트라', '스터바이 코리도라스', '스포티드 메티니스', '시아미즈 알지이터', '실버 구라미', '실버 바브', '실버 샤크', 
              '실버팁 테트라', '아이스블루 시클리드', '안시', '알리 시클리드', '알비노 코리도라스', '앰버 테트라', '엔젤피쉬', '엠페러 테트라', '오데사 바브', '오란다 금붕어', '오셀라투스', '왁 플래티', 
              '유금 금붕어', '인디언 복어', '저먼 라미레지', '제브라 다니오', '진주린 금붕어', '찬다랑가', '체리 바브', '초록 복어', '초콜릿 구라미', '카디널 테트라', '컨빅트 시클리드', '코메트 금붕어', 
              '코멧 플래티', '코발트 블루 구라미', '콜롬비아 테트라', '콩고 테트라', '쿨리 로치', '크라운 로치', '크라운 킬리피쉬', '키싱 구라미', '키티 테트라', '턱시도 플래티', '툭눈 금붕어', 
              '트로페우스 드보이시', '파키스탄 로치', '판다 플래티', '팔바 레인보우', '팬더 로치', '팬더 코리도라스', '펄 구라미', '페퍼드 코리도라스', '펭귄 테트라', '풍선 몰리', '프론토사', 
              '프리스텔라 리들레이', '플라워혼', '플라캇 베타', '플레임 테트라', '피그메우스 코리도라스', '피그미 구라미', '피콕 시클리드', '하스타투스 코리도라스', '하프문 베타', '허니 구라미', '혈앵무']


# 랜덤 이미지 선택
num_images = len(test_datasets)
rand_idx = random.randint(0, num_images-1)
image, label = list(test_datasets.take(1))[0]

for image_batch, label_batch in test_datasets.take(1):
    # 모델 예측
    predictions = model_res.predict(image_batch) # 64개 한꺼번에 예측
    
    for i in range(len(label_batch)):
        predicted_classes = np.argsort(predictions[i])[::-1][:5]
        print(f'정답 :  {class_name[np.argmax(label_batch[i])]}\nTop 5 물고기 :')
        for idx in predicted_classes:
            print(f" - {class_name[idx]}: {predictions[i][idx]}")


KeyboardInterrupt: ignored

In [7]:
#모델 저장하는 코드

model_res.save('drive/MyDrive/models/model5')

In [13]:
#ngrok설치
!pip install pyngrok==4.1.1
!pip install flask_ngrok

#서버 코드
import numpy as np
from io import BytesIO 
from flask_ngrok import run_with_ngrok
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from flask import Flask, request, jsonify
import matplotlib.pyplot as plt


class_name = ['가드네리 킬리피쉬', '고도비', '골든 구라미', '골든 바브', '골든 알지이터', '구피', '그린 네온 테트라', '글라스 블러드 핀 테트라', '글라스캣', '글로라이트 다니오', 
              '글로라이트 테트라', '나비 비파', '난주 금붕어', '남미 복어', '네온 드워프 레인보우', '네온 테트라', '뉴기니아 레인보우', '다람쥐 시클리드', '다이아몬드 테트라', 
              '드워프 구라미', '디스커스', '라스보라 갤럭시', '라스보라 에메랄드', '라스보라 쿠보타이', '라스보라 헤테로몰파', '라스보라 헹겔리', '라이어테일 몰리', '러미노즈 테트라', 
              '레드 레인보우', '레드 플래티', '레몬 테트라', '레오파드 다니오', '레인보우 샤크', '로즈라인 바브', '로지 바브', '리코리스 구라미', '마다가스카르 레인보우', '메티니스', 
              '메티니스 블랙바', '몽크호샤 코스테', '몽크호샤(레드아이 테트라)', '물티', '미키마우스 플래티', '바나나 시클리드', '백설 시클리드', '백운산', '밴디드 구라미', '밴디드 레인보우', 
              '밴디드 레포리너스', '버터플라이 레인보우', '벌룬 라미레지', '범블비 플래티', '베일테일 베타', '보세마니 레인보우', '볼리비안 라미레지', '브론즈 코리도라스', '브리샤르디', 
              '블랙 네온 테트라', '블랙 테트라(컬러 테트라)', '블루 레인보우', '블루 스팟 구라미', '블루 제브라 시클리드', '블루킹 테트라', '비너스투스 시클리드', '뽀본데타 레인보우', '샤페 테트라', 
              '선셋 구라미', '선셋 플래티', '세베럼', '셀핀 몰리', '소드테일', '수마트라', '스터바이 코리도라스', '스포티드 메티니스', '시아미즈 알지이터', '실버 구라미', '실버 바브', '실버 샤크', 
              '실버팁 테트라', '아이스블루 시클리드', '안시', '알리 시클리드', '알비노 코리도라스', '앰버 테트라', '엔젤피쉬', '엠페러 테트라', '오데사 바브', '오란다 금붕어', '오셀라투스', '왁 플래티', 
              '유금 금붕어', '인디언 복어', '저먼 라미레지', '제브라 다니오', '진주린 금붕어', '찬다랑가', '체리 바브', '초록 복어', '초콜릿 구라미', '카디널 테트라', '컨빅트 시클리드', '코메트 금붕어', 
              '코멧 플래티', '코발트 블루 구라미', '콜롬비아 테트라', '콩고 테트라', '쿨리 로치', '크라운 로치', '크라운 킬리피쉬', '키싱 구라미', '키티 테트라', '턱시도 플래티', '툭눈 금붕어', 
              '트로페우스 드보이시', '파키스탄 로치', '판다 플래티', '팔바 레인보우', '팬더 로치', '팬더 코리도라스', '펄 구라미', '페퍼드 코리도라스', '펭귄 테트라', '풍선 몰리', '프론토사', 
              '프리스텔라 리들레이', '플라워혼', '플라캇 베타', '플레임 테트라', '피그메우스 코리도라스', '피그미 구라미', '피콕 시클리드', '하스타투스 코리도라스', '하프문 베타', '허니 구라미', '혈앵무']


app = Flask(__name__)
model = tf.keras.models.load_model('drive/MyDrive/models/model6')

@app.route('/predict', methods=['POST'])
def predict():
    # 이미지 파일을 업로드 받음
    img = BytesIO(request.files['image'].read())

    # 이미지 전처리
    img = image.load_img(img, target_size=(250, 250))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # 모델 예측
    predictions = model.predict(x)
    predicted_classes = np.argsort(predictions[0])[::-1][:5]  # 사진 예측에서 값이 큰 순서대로 상위 5개 인덱스 가져오기

    #colab에서 결과 확인 & 물고기 이름 배열에 추가
    print("Top 5 물고기 : ")
    rank = []
    for idx in predicted_classes: #상위 5개 인덱스
        rank.append(class_name[idx]) #물고기 이름을 결과에 추가
        print(" - ", class_name[idx], ": ", predictions[0][idx])


    #예측 결과 반환
    print("결과: ",rank[0], rank[1],rank[2]) 
    return jsonify({'prediction1': rank[0], 'prediction2' : rank[1], 'prediction3' : rank[2], 'prediction4' : rank[3], 'prediction5' : rank[4]})




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=dac8b8cc62b294791c9079b71a4140f845d00e832955aa941e754bb751b85b8e
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
#서버 실행

run_with_ngrok(app)
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cb64-34-126-70-206.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
1/1 [==============================] - 1s 1s/step


INFO:werkzeug:127.0.0.1 - - [15/May/2023 13:45:52] "POST /predict HTTP/1.1" 200 -


Top 5 물고기 : 
 -  플라캇 베타 :  0.52143574
 -  블랙 테트라(컬러 테트라) :  0.1700926
 -  풍선 몰리 :  0.12727787
 -  피콕 시클리드 :  0.12257902
 -  콜롬비아 테트라 :  0.03280408
결과:  플라캇 베타 블랙 테트라(컬러 테트라) 풍선 몰리
1/1 [==============================] - 0s 25ms/step


...

Top 5 물고기 : 
 -  피그메우스 코리도라스 :  0.98729104
 -  시아미즈 알지이터 :  0.008839134
 -  하스타투스 코리도라스 :  0.0026999644
 -  블랙 네온 테트라 :  0.00038503727
 -  라스보라 헹겔리 :  0.00023156742
결과:  피그메우스 코리도라스 시아미즈 알지이터 하스타투스 코리도라스
1/1 [==============================] - 0s 23ms/step


INFO:werkzeug:127.0.0.1 - - [15/May/2023 13:51:16] "POST /predict HTTP/1.1" 200 -


Top 5 물고기 : 
 -  블루 스팟 구라미 :  0.99950755
 -  혈앵무 :  0.00033473957
 -  컨빅트 시클리드 :  9.6407945e-05
 -  엔젤피쉬 :  2.9381457e-05
 -  플라캇 베타 :  1.5538262e-05
결과:  블루 스팟 구라미 혈앵무 컨빅트 시클리드
1/1 [==============================] - 0s 25ms/step


INFO:werkzeug:127.0.0.1 - - [15/May/2023 13:51:50] "POST /predict HTTP/1.1" 200 -


Top 5 물고기 : 
 -  블루킹 테트라 :  0.9564995
 -  글라스 블러드 핀 테트라 :  0.021057885
 -  레몬 테트라 :  0.0084320465
 -  샤페 테트라 :  0.0056800796
 -  실버팁 테트라 :  0.0029920184
결과:  블루킹 테트라 글라스 블러드 핀 테트라 레몬 테트라
